In [9]:
import numpy as np
import keras
from keras import backend as K
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Activation, Dropout, GlobalAveragePooling2D

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

In [3]:
CLASSES = 2 # missing pulley or perfect pulley

baseModel = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = baseModel.output

#Test the SVM or DNN
# Change the Flatten layer,
x = GlobalAveragePooling2D()(x)

# Change the number of Dense layer and number of neuron
# Make the relu6 activation function and test it.
x = Dense(1024, activation='relu')(x)

# Change the Dropout ratio
x = Dropout(0.5)(x)
#x = Dense(1024, activation='relu')(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.5)(x)
preds = Dense(CLASSES, activation='softmax')(x)

model = Model(inputs=baseModel.input, outputs=preds)
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.


In [27]:
for i, layer in enumerate(model.layers):
    print("{}: {}".format(i, layer))

0: <keras.engine.input_layer.InputLayer object at 0x7f489e940d50>
1: <keras.layers.convolutional.ZeroPadding2D object at 0x7f48dc08f1d0>
2: <keras.layers.convolutional.Conv2D object at 0x7f48dc0eead0>
3: <keras.layers.normalization.BatchNormalization object at 0x7f489e957310>
4: <keras.layers.advanced_activations.ReLU object at 0x7f489e111350>
5: <keras.layers.convolutional.DepthwiseConv2D object at 0x7f489e12bfd0>
6: <keras.layers.normalization.BatchNormalization object at 0x7f489e957490>
7: <keras.layers.advanced_activations.ReLU object at 0x7f48947c5610>
8: <keras.layers.convolutional.Conv2D object at 0x7f48947c5dd0>
9: <keras.layers.normalization.BatchNormalization object at 0x7f48946fcf10>
10: <keras.layers.advanced_activations.ReLU object at 0x7f4894738990>
11: <keras.layers.convolutional.ZeroPadding2D object at 0x7f4894738d90>
12: <keras.layers.convolutional.DepthwiseConv2D object at 0x7f48946eae10>
13: <keras.layers.normalization.BatchNormalization object at 0x7f48945fff10>
14:

In [26]:
# Adjust the range of the network 
for layer in model.layers[:87]:
    layer.trainable = False

for layer in model.layers[87:]:
    layer.trainable = True

In [28]:
# Stratified selection of training set and validation set, and test set
trainDataGen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split = 0.3)

# Change the batch_size
trainGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            #color_mode = 'rgb',
                                            class_mode = 'categorical',
                                            subset = 'training',
                                            shuffle = True)

validGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                           target_size = (224, 224),
                                           batch_size = 64,
                                           class_mode = 'categorical',
                                           subset = 'validation')

Found 15860 images belonging to 2 classes.
Found 6795 images belonging to 2 classes.


In [31]:
FILEPATH = '/home/dongjun/djplace/samples/'
EPOCHS = 2 #10, or early stopping
LEARNINGRATE = 0.001 # decay the learning rate
DECAYRATE = LEARNINGRATE // EPOCHS
opt = Adam(lr = LEARNINGRATE, decay=DECAYRATE)

# ADD FOCAL LOSS
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
checkpoint = ModelCheckpoint('weights/model.hdf5', monitor = 'val_acc', mode = 'min', save_weights_only = False, save_best_only = True, verbose=1)
tfBoard = TensorBoard(log_dir = 'logs/')
callbacksList = [checkpoint, tfBoard]

In [33]:
history =model.fit_generator(generator=trainGen,
                             steps_per_epoch = trainGen.n // trainGen.batch_size,
                             validation_data = validGen,
                             validation_steps = validGen.n // validGen.batch_size,
                             callbacks = callbacksList,
                             epochs = EPOCHS)

Epoch 1/2
247/247 [==============================] - 1648s 7s/step - loss: 0.0763 - accuracy: 0.9701 - val_loss: 3.4251 - val_accuracy: 0.6263
Epoch 2/2
247/247 [==============================] - 1639s 7s/step - loss: 0.0611 - accuracy: 0.9742 - val_loss: 3.2450 - val_accuracy: 0.6601


In [ ]:
inference = keras.models.load_model('./model.hdf5')
def preprocessImage(img):
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_NEAREST)
    return img